# Init Spark

In [6]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

from pathlib import Path

In [70]:
spark


ConnectionRefusedError: [Errno 61] Connection refused

In [69]:
conf = SparkConf() \
    .setMaster("local[*]") \
    .setAppName("test") \
    .set(
        "spark.jars",
        "file:/Users/vince/protonx-DE/day_5/lib/gcs-connector-hadoop3-latest.jar"
    )

spark_ctx = SparkContext(conf=conf).getOrCreate()

hadoop_conf = spark_ctx._jsc.hadoopConfiguration()
hadoop_conf.set(
    "fs.AbstractFileSystem.gs.impl",
    "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS"
)
hadoop_conf.set(
    "fs.gs.impl",
    "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem"
)
    
spark = SparkSession.builder \
    .config(conf=spark_ctx.getConf()) \
    .getOrCreate()

ConnectionRefusedError: [Errno 61] Connection refused

.master("local[*]").  Nếu chạy local. Dấu * tượng trưng cho ý spark được phép dùng hết tài nguyên của máy ( CPU/RAM ) để xử lý 

.master("yarn"). Nếu chạy trên cluster YARN


Có thể thêm config vào. Một số config để connect với s3

  - .config("spark.jars", "aws-sdk-java-2.17.81.jar") 

  - .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
  
  - .config("spark.hadoop.fs.s3a.access.key", "<your_access_key_id>")
  
  - .config("spark.hadoop.fs.s3a.secret.key", "<your_secret_access_key>")

In [4]:
!pyspark --version and spark-shell --version 

23/09/19 21:54:21 WARN Utils: Your hostname, vinces-MBP.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
23/09/19 21:54:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 17.0.2
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.


# Read File

In [17]:
# data_storage = "gs://aws-review-data/amazon_us_reviews-train-00000-of-00005.parquet"
data_storage = "./day_5_data/amazon_us_reviews-train-00000-of-00005.parquet"
data_files = [ str(file) for file in Path("./day_5_data").rglob("*.parquet") ]

df = spark.read.parquet(*data_files)

In [18]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: long (nullable = true)
 |-- verified_purchase: long (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [19]:
df.schema.jsonValue()

{'type': 'struct',
 'fields': [{'name': 'marketplace',
   'type': 'string',
   'nullable': True,
   'metadata': {}},
  {'name': 'customer_id', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'review_id', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'product_id', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'product_parent',
   'type': 'string',
   'nullable': True,
   'metadata': {}},
  {'name': 'product_title',
   'type': 'string',
   'nullable': True,
   'metadata': {}},
  {'name': 'product_category',
   'type': 'string',
   'nullable': True,
   'metadata': {}},
  {'name': 'star_rating', 'type': 'integer', 'nullable': True, 'metadata': {}},
  {'name': 'helpful_votes',
   'type': 'integer',
   'nullable': True,
   'metadata': {}},
  {'name': 'total_votes', 'type': 'integer', 'nullable': True, 'metadata': {}},
  {'name': 'vine', 'type': 'long', 'nullable': True, 'metadata': {}},
  {'name': 'verified_purchase',
   'type': 'long',
   

In [20]:
df.show(2)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-----------+
|         US|   47539447|R1HQ2TY3LSFU0R|B00JA7GSVO|     658880856|Spring Notion Boy...|         Apparel|          4|            0|          0|   0|                1|     Four Stars|Good quality. Rea...| 2014-12-13|
|         US|   44940202| R49AHLER1ILGZ|B00JA7GSO6|     658880856|Spring Notion Boy...|         Apparel|          5|            0|          

# Read Spark with Schema

In [21]:
from pyspark.sql import types

In [28]:
schema = types.StructType([
    types.StructField('marketplace', types.StringType(), True),
    types.StructField('customer_id', types.StringType(), True),
    types.StructField('review_id', types.StringType(), True),
    types.StructField('product_id', types.StringType(), True),
    types.StructField('product_parent', types.StringType(), True),
    types.StructField('product_category', types.StringType(), True),
    types.StructField('star_rating', types.IntegerType(), True),
    types.StructField('helpful_votes', types.IntegerType(), True),
    types.StructField('total_votes', types.IntegerType(), True),
    types.StructField('vine', types.LongType(), True),
    types.StructField('verified_purchase', types.LongType(), True),
    types.StructField('review_headline', types.StringType(), True),
    types.StructField('review_body', types.StringType(), True),
    types.StructField('review_date', types.StringType(), True)
])

In [29]:
df = spark.read \
    .schema(schema) \
    .parquet(data_storage)

In [30]:
df.show(2)

+-----------+-----------+--------------+----------+--------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|         Apparel|          4|            0|          0|   0|                1|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|         Apparel|          5|            1|          2|   0|                1|Favorite for wint...|I love this dress...| 2014-03-04|
+----

# Làm việc với cột ( columns ) 

In [31]:
import pyspark.sql.functions as F

In [32]:
df = df.withColumn(
    'star_rating', df['star_rating'].cast(types.FloatType())) # Transform Col 
df = df.withColumn(
    'review_date', F.to_date(F.col("review_date")))
# df.drop('product_parent') # Drop column

df = df.withColumnRenamed("product_parent","productParent")

In [33]:
df.select("star_rating","productParent").describe().show()

+-------+-----------------+-------------------+
|summary|      star_rating|      productParent|
+-------+-----------------+-------------------+
|  count|          1440000|            1440000|
|   mean|4.035313194444444|4.984089286230021E8|
| stddev|1.318571677617217|2.895259558244379E8|
|    min|              1.0|          100001307|
|    max|              5.0|          999997291|
+-------+-----------------+-------------------+



In [34]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- productParent: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: float (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: long (nullable = true)
 |-- verified_purchase: long (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



# Pyspark và SQL

### SELECT

In [35]:
df.select("*").show(1)

+-----------+-----------+--------------+----------+-------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|productParent|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+-------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|     24485154|         Apparel|        4.0|            0|          0|   0|                1|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
+-----------+-----------+--------------+----------+-------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
only showi

In [39]:
df.select(
    "customer_id",
    "product_id",
    "review_headline",
    "star_rating",
    "helpful_votes").show(2)

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|   32158956|B01KL6O72Y|★ THESE REALLY DO...|        4.0|            0|
|    2714559|B01ID3ZS5W|Favorite for wint...|        5.0|            1|
+-----------+----------+--------------------+-----------+-------------+
only showing top 2 rows



In [40]:
df.select("product_id").distinct().show(5)

+----------+
|product_id|
+----------+
|B014110CIS|
|B013RTQQXK|
|B013GVHRZ0|
|B012HKN4B2|
|B012C86TZW|
+----------+
only showing top 5 rows



In [41]:
df.select(
    F.col("product_id"),
    F.col("customer_id"),
).distinct().show()

+----------+-----------+
|product_id|customer_id|
+----------+-----------+
|B01HDXFZK6|   25482800|
|B014EDMV32|   26780399|
|B014397P24|     411718|
|B013QO8ZVM|   10931103|
|B013PU4ZE8|     666985|
|B013OTEXCO|   31758998|
|B013MB7TO8|    8927659|
|B013F5IY0O|   42742751|
|B013F4GPD8|   30057578|
|B013D6LSC6|     110520|
|B0138W9IEU|   11860569|
|B0133Y39LQ|   23202878|
|B0131W39I8|   31464343|
|B0131D4NKU|   21248122|
|B012XW6Z1U|   41961605|
|B012VO80FE|   21300706|
|B012TNCVC0|    1989995|
|B012P0CU6Y|    1876472|
|B012MADJKS|     138879|
|B012M9Y33Q|    1662872|
+----------+-----------+
only showing top 20 rows



### WHERE

In [43]:
df.select(
    F.col("customer_id"),
    F.col("product_id"),
    F.col("review_headline"),
    F.col("star_rating"),
    F.col("helpful_votes")
).filter("star_rating >= 5").show()

+-----------+----------+--------------------+-------------+
|customer_id|product_id|     review_headline|helpful_votes|
+-----------+----------+--------------------+-------------+
|    2714559|B01ID3ZS5W|Favorite for wint...|            1|
|   12608825|B01I497BGY|Great Socks for t...|            0|
|   25482800|B01HDXFZK6|          Slick hat!|            0|
|    9310286|B01G6MBEBY|I would do it again!|            0|
|   26631939|B01FWRXN0Y|          Five Stars|            0|
|   48785098|B01EXNH1HE|            Love it!|            0|
|   29355866|B01DXHX81O|          Five Stars|            0|
|   17685865|B01BOKOL4A|The Jockey Women'...|            0|
|   19787539|B01B3Q4Q0O|          Five Stars|            0|
|   44196725|B01ADDSL9U|          Five Stars|            0|
|   45510794|B019P1X5XI|best ever4 for me...|            0|
|   42025740|B019MDXIXG|          Five Stars|            0|
|   27097189|B019438FEG|          Five Stars|            0|
|   30225894|B016VIU0QI|   Awesome leggi

In [82]:
df.select(
    F.col("customer_id"),
    F.col("product_id"),
    F.col("review_headline"),
    F.col("star_rating"),
    F.col("helpful_votes")
).filter(F.col("star_rating") >= 4).show()

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|   32158956|B01KL6O72Y|★ THESE REALLY DO...|          4|            0|
|    2714559|B01ID3ZS5W|Favorite for wint...|          5|            1|
|   12608825|B01I497BGY|Great Socks for t...|          5|            0|
|   25482800|B01HDXFZK6|          Slick hat!|          5|            0|
|    9310286|B01G6MBEBY|I would do it again!|          5|            0|
|   26631939|B01FWRXN0Y|          Five Stars|          5|            0|
|   48785098|B01EXNH1HE|            Love it!|          5|            0|
|   39548589|B01E7OL09O|         Three Stars|          4|            0|
|   29355866|B01DXHX81O|          Five Stars|          5|            0|
|   17685865|B01BOKOL4A|The Jockey Women'...|          5|            0|
|   19787539|B01B3Q4Q0O|          Five Stars|          5|       

In [45]:
condition = (F.col("star_rating") >= 4) & (F.col("helpful_votes") > 10)

df.select(
    F.col("customer_id"),
    F.col("product_id"),
    F.col("review_headline"),
    F.col("star_rating"),
    F.col("helpful_votes")
).filter(condition).show()

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|   10090640|B0147UWNUI|   Plus size review!|        5.0|           12|
|   43100575|B0145JUDJY|SO Awesome (and a...|        5.0|           13|
|     107041|B01411YLLM|    I love this coat|        5.0|           15|
|   17162238|B0140UFQKE|This is an extrem...|        5.0|           30|
|     453792|B013QTWRAC|          Five Stars|        5.0|           51|
|    2089228|B013P4QQVO|            Love it!|        5.0|           29|
|     114396|B013P29CBM|Colors look like ...|        5.0|          148|
|   51748675|B013OUQEUM|Very beautiful fa...|        4.0|           23|
|   52045345|B013OUQ8PS|Flowy design adds...|        4.0|           11|
|   20036398|B013LFUL8G|Quality product-E...|        5.0|           16|
|   50038974|B013KVV1NA|Cute hoodie made ...|        5.0|       

### ORDER BY

In [46]:
df.select(
    F.col("customer_id"),
    F.col("product_id"),
    F.col("review_headline"),
    F.col("star_rating"),
    F.col("helpful_votes")
).filter(F.col("star_rating") >= 4).orderBy("star_rating").show()

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|   23621953|B00K0A8AB6|          Four Stars|        4.0|            0|
|   49337986|B00KU3CXJS|          Four Stars|        4.0|            0|
|      18254|B00QWCGFSQ|          Four Stars|        4.0|            0|
|   13226943|B00KU2I9FG|      Cute and Sexy!|        4.0|            8|
|   11058879|B00OLHIW8U|          Four Stars|        4.0|            0|
|   28966347|B00KU3BKJW|          Four Stars|        4.0|            0|
|   10822606|B00QWAOF5S|          Four Stars|        4.0|            0|
|   10867285|B00KU2WEMU|          Four Stars|        4.0|            0|
|    2426550|B00N53FLK8|Beautiful and war...|        4.0|            4|
|   39561738|B00KU2I9VU|Great lift, paddi...|        4.0|            0|
|   50240384|B00QWBKLAA|This is really cu...|        4.0|       

In [47]:
df.select(
    F.col("customer_id"),
    F.col("product_id"),
    F.col("review_headline"),
    F.col("star_rating"),
    F.col("helpful_votes")
).filter(F.col("star_rating") >= 4).sort("star_rating").show()

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|   30740886|B014WBIGKI|but is a nice swe...|        4.0|            0|
|   11058879|B00OLHIW8U|          Four Stars|        4.0|            0|
|   49337986|B00KU3CXJS|          Four Stars|        4.0|            0|
|    7231775|B00OLHFPJY|             Awesome|        4.0|            0|
|      18254|B00QWCGFSQ|          Four Stars|        4.0|            0|
|   41233109|B00OLHIRBW|            Torn....|        4.0|            0|
|   13226943|B00KU2I9FG|      Cute and Sexy!|        4.0|            8|
|   36429969|B00OLHOZX6|                Good|        4.0|            0|
|    2426550|B00N53FLK8|Beautiful and war...|        4.0|            4|
|     113492|B00OLHIQH2|          Four Stars|        4.0|            0|
|   28966347|B00KU3BKJW|          Four Stars|        4.0|       

In [48]:
( 
    df.select(
            F.col("customer_id"),
            F.col("product_id"),
            F.col("review_headline"),
            F.col("star_rating"),
            F.col("helpful_votes"))
        .filter(F.col("star_rating") >= 4)
        .sort(
            F.col("star_rating").desc()
        ).show()
)

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|   13117898|B00K0A8AB6|Cute gift for lit...|        5.0|            0|
|   29182724|B00UH0X10E|          Five Stars|        5.0|            0|
|   22116819|B014MSSP66|          Five Stars|        5.0|            0|
|   22787620|B00UGYOA4C|          Five Stars|        5.0|            0|
|   17366243|B00OLHIZXW|... return to get...|        5.0|            0|
|   43848429|B00UH0GQOM|          Five Stars|        5.0|            0|
|     643748|B014E1C364|          Five Stars|        5.0|            0|
|   12285020|B00UH4P1SA|          Five Stars|        5.0|            0|
|    3377581|B00QWCWO5Y|          Five Stars|        5.0|            0|
|   31431851|B00UGZ3ZF6|Great shirt! Very...|        5.0|            0|
|   47482829|B014M5N1WW|Arrived quickly a...|        5.0|       

In [95]:
( 
    df.select(
            F.col("customer_id"),
            F.col("product_id"),
            F.col("review_headline"),
            F.col("star_rating"),
            F.col("helpful_votes"))
        .filter(F.col("star_rating") >= 4)
        .sort(
            F.desc("star_rating") 
        ).show()
)

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|   22116819|B014MSSP66|          Five Stars|        5.0|            0|
|     643748|B014E1C364|          Five Stars|        5.0|            0|
|   47482829|B014M5N1WW|Arrived quickly a...|        5.0|            0|
|    2714559|B01ID3ZS5W|Favorite for wint...|        5.0|            1|
|    4105777|B014L79H8I|          Great Gift|        5.0|            0|
|   25482800|B01HDXFZK6|          Slick hat!|        5.0|            0|
|     502031|B014K9ETKS|          Five Stars|        5.0|            3|
|   26631939|B01FWRXN0Y|          Five Stars|        5.0|            0|
|   44617615|B014K3PHXW|          Five Stars|        5.0|            0|
|   46404187|B014K01GSU|Excellent RFID Wa...|        5.0|            0|
|   26780399|B014EDMV32|            Love it!|        5.0|       

###  Limit 

In [86]:
( 
    df.select(
            F.col("customer_id"),
            F.col("product_id"),
            F.col("review_headline"),
            F.col("star_rating"),
            F.col("helpful_votes"))
        .filter(F.col("star_rating") >= 4)
        .sort(
            F.col("star_rating").desc()
        )
        .limit(1)
        .show()
)

+-----------+----------+--------------------+-----------+-------------+
|customer_id|product_id|     review_headline|star_rating|helpful_votes|
+-----------+----------+--------------------+-----------+-------------+
|    2714559|B01ID3ZS5W|Favorite for wint...|          5|            1|
+-----------+----------+--------------------+-----------+-------------+



### GROUP BY 

In [53]:
( 
    df.select(
            F.col("product_id"),
            F.col("star_rating"),
            F.col("helpful_votes")
    )
    .groupBy("product_id")
    .mean()
    .show()
)

+----------+----------------+------------------+
|product_id|avg(star_rating)|avg(helpful_votes)|
+----------+----------------+------------------+
|B014110CIS|             2.0|               0.0|
|B013RTQQXK|             5.0|               0.0|
|B013GVHRZ0|             3.0|               0.0|
|B012HKN4B2|             3.0|               1.0|
|B012C86TZW|             5.0|               0.0|
|B012698SU6|             5.0|               2.0|
|B0120RNKE8|             3.0|               1.0|
|B0120OP7NI|             5.0|               0.0|
|B011XDAW3W|             3.0|               0.0|
|B011WELXAI|             5.0|               0.0|
|B011PJXIE4|             5.0|               0.0|
|B011NNTFDK|             1.0|               0.0|
|B011NF7YUY|             5.0|               2.0|
|B011M97GR2|             5.0|               0.0|
|B011M5NSZ0|             5.0|               7.0|
|B011JG86VI|             5.0|               0.0|
|B011J5WPBQ|             5.0|               5.0|
|B011IF6G3K|        

In [99]:
( 
    df.select(
            F.col("product_id"),
            F.col("review_headline"),
            F.col("star_rating"),
            F.col("helpful_votes")
        )
        .groupBy("product_id")
        .mean()
        .show()
)

+----------+----------------+------------------+
|product_id|avg(star_rating)|avg(helpful_votes)|
+----------+----------------+------------------+
|B014110CIS|             2.0|               0.0|
|B013RTQQXK|             5.0|               0.0|
|B013GVHRZ0|             3.0|               0.0|
|B012HKN4B2|             3.0|               1.0|
|B012C86TZW|             5.0|               0.0|
|B012698SU6|             5.0|               2.0|
|B0120RNKE8|             3.0|               1.0|
|B0120OP7NI|             5.0|               0.0|
|B011XDAW3W|             3.0|               0.0|
|B011WELXAI|             5.0|               0.0|
|B011PJXIE4|             5.0|               0.0|
|B011NNTFDK|             1.0|               0.0|
|B011NF7YUY|             5.0|               2.0|
|B011M97GR2|             5.0|               0.0|
|B011M5NSZ0|             5.0|               7.0|
|B011JG86VI|             5.0|               0.0|
|B011J5WPBQ|             5.0|               5.0|
|B011IF6G3K|        


[Stage 61:>                                                         (0 + 1) / 1]



In [54]:
( 
    df.select(
            F.col("product_id"),
            F.col("review_headline"),
            F.col("star_rating"),
            F.col("helpful_votes")
        )
        .groupBy("product_id")
        .agg(
            F.min("star_rating").alias("min_star_rating"),
            F.max("helpful_votes").alias("max_helpful_votes")
        )
        .show()
)

+----------+---------------+-----------------+
|product_id|min_star_rating|max_helpful_votes|
+----------+---------------+-----------------+
|B014110CIS|            2.0|                0|
|B013RTQQXK|            5.0|                0|
|B013GVHRZ0|            3.0|                0|
|B012HKN4B2|            3.0|                1|
|B012C86TZW|            5.0|                0|
|B012698SU6|            5.0|                3|
|B0120RNKE8|            3.0|                1|
|B0120OP7NI|            5.0|                0|
|B011XDAW3W|            3.0|                0|
|B011WELXAI|            5.0|                0|
|B011PJXIE4|            5.0|                0|
|B011NNTFDK|            1.0|                0|
|B011NF7YUY|            5.0|                2|
|B011M97GR2|            5.0|                0|
|B011M5NSZ0|            5.0|                7|
|B011JG86VI|            5.0|                0|
|B011J5WPBQ|            5.0|                5|
|B011IF6G3K|            5.0|                0|
|B011HWM0GQ| 

# User Define Function ( UDF )

In [55]:
def convert_case(string):
    return string.upper()

convert_case_udf = F.udf(convert_case, returnType=types.StringType())

In [57]:
( 
    df
    .withColumn('review_headline_uppercase', convert_case_udf(F.col("review_headline")))
    .select('review_headline_uppercase','review_headline') 
    .show()
)

+-------------------------+--------------------+
|review_headline_uppercase|     review_headline|
+-------------------------+--------------------+
|     ★ THESE REALLY DO...|★ THESE REALLY DO...|
|     FAVORITE FOR WINT...|Favorite for wint...|
|     GREAT SOCKS FOR T...|Great Socks for t...|
|               SLICK HAT!|          Slick hat!|
|     I WOULD DO IT AGAIN!|I would do it again!|
|               FIVE STARS|          Five Stars|
|                 LOVE IT!|            Love it!|
|              THREE STARS|         Three Stars|
|               FIVE STARS|          Five Stars|
|         NOT MY FAVORITE.|    Not my favorite.|
|     THE JOCKEY WOMEN'...|The Jockey Women'...|
|               FIVE STARS|          Five Stars|
|               FIVE STARS|          Five Stars|
|     BEST EVER4 FOR ME...|best ever4 for me...|
|               FIVE STARS|          Five Stars|
|               FIVE STARS|          Five Stars|
|               FOUR STARS|          Four Stars|
|        AWESOME LEG

# Spark SQL

In [59]:
df.createOrReplaceTempView("df")
# df.registerTempTable("df")

In [117]:
spark.sql(" SELECT * from df ").show()

+-----------+-----------+--------------+----------+-------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|productParent|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+-------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|     24485154|         Apparel|        4.0|            0|          0|   0|                1|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|    363128556|         Apparel|        5.0|            1|          2|   0|                1|Favorite for wint...|I love this dress...| 2014-03-04|
|         

In [119]:
query = """

SELECT product_id, MAX(star_rating), MIN(helpful_votes)
FROM df 
WHERE star_rating >=4 and helpful_votes >= 10
GROUP BY product_id
LIMIT 10

""" 
spark.sql(query).show()

+----------+----------------+------------------+
|product_id|max(star_rating)|min(helpful_votes)|
+----------+----------------+------------------+
|B00MMMSNMQ|             5.0|                12|
|B00MF78G84|             4.0|                13|
|B00M8I0B1A|             5.0|                13|
|B00LY1YPAK|             5.0|                17|
|B00LXTN63K|             4.0|                33|
|B00LXG1EM8|             5.0|                27|
|B00LWG59AW|             5.0|                55|
|B00LVZN3F2|             5.0|                11|
|B00LSXELJY|             5.0|                42|
|B00LQMW4YQ|             5.0|                10|
+----------+----------------+------------------+



# Ghi kết quả ra file parquet 

In [60]:
query = """

SELECT product_id, MAX(star_rating), MIN(helpful_votes)
FROM df 
WHERE star_rating >=4 and helpful_votes >= 10
GROUP BY product_id

""" 

spark.sql(query).write.parquet(
    "./day_5_data/report-000.parquet",
    mode = "overwrite"
)

#Dùng  write.partitionBy(col).parquet để partition

In [61]:
result_df = spark.read.parquet("./day_5_data/report-000.parquet")

In [62]:
result_df.show()

+----------+----------------+------------------+
|product_id|max(star_rating)|min(helpful_votes)|
+----------+----------------+------------------+
|B011DU3MU0|             5.0|                11|
|B010NG5OAG|             5.0|                45|
|B00ZQ3U93O|             5.0|                24|
|B00Y0M6878|             5.0|                11|
|B00XVBSBGK|             4.0|                15|
|B00XKFVL32|             4.0|                11|
|B00XIZ1RI8|             5.0|               105|
|B00WW145WY|             4.0|                10|
|B00WPKMQS2|             4.0|                14|
|B00WB6FCE0|             5.0|                32|
|B00WAKT6BM|             5.0|                21|
|B00VTVDCRW|             5.0|               164|
|B00UN3924Y|             5.0|                18|
|B011LTPJWW|             5.0|                76|
|B010NUNZW6|             5.0|                15|
|B010EA4KOM|             5.0|                10|
|B00ZZATRZE|             4.0|                18|
|B00ZA1G62A|        

# Ví dụ thực tế.

Đếm các comment tốt cho theo sản phẩm + mỗi ngày.

Tiêu chí tốt: 

    - star_rating >= 4 

Tiêu chí lọc comment 

    - helpful_votes >= 10


In [1]:
from pathlib import Path

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types

spark = ( 
    SparkSession.builder
            .master("local[*]") 
            .appName('Aggregate Good Comment')
            .getOrCreate()
)

schema = types.StructType([
    types.StructField('marketplace', types.StringType(), True),
    types.StructField('customer_id', types.StringType(), True),
    types.StructField('review_id', types.StringType(), True),
    types.StructField('product_id', types.StringType(), True),
    types.StructField('product_parent', types.StringType(), True),
    types.StructField('product_category', types.StringType(), True),
    types.StructField('star_rating', types.IntegerType(), True),
    types.StructField('helpful_votes', types.IntegerType(), True),
    types.StructField('total_votes', types.IntegerType(), True),
    types.StructField('vine', types.LongType(), True),
    types.StructField('verified_purchase', types.LongType(), True),
    types.StructField('review_headline', types.StringType(), True),
    types.StructField('review_body', types.StringType(), True),
    types.StructField('review_date', types.StringType(), True)
])

# data_storage = "gs://aws-review-data/read/amazon_us_reviews-train-00000-of-00005.parquet"
data_storage = [ 
    str(file) for file in Path("./day_5_data").rglob("amazon_us_reviews-*.parquet") 
]
data_write = "./day_5_data/report-count"

df = spark.read.parquet(*data_storage)
df = df.withColumn('review_date', F.to_date(F.col("review_date")))

( 
    df.select(
            F.col("product_id"),
            F.col("review_date"),
            F.col("star_rating"),
            F.col("helpful_votes")
        )
        .where((F.col("star_rating") >= 4 ) & (F.col("helpful_votes") >= 10 ))
        .groupBy(["product_id","review_date"])
        .count()
        .write.parquet(data_write,mode = "overwrite")
)


# df.createOrReplaceTempView("df")
# query = """

# SELECT product_id,review_date,COUNT(star_rating)
# FROM df 
# WHERE star_rating >=4 and helpful_votes >= 10
# GROUP BY product_id
# """ 

# spark.sql(query).write.parquet(data_write,
#                         mode = "overwrite")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/21 20:58:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
result_df = spark.read.parquet("./day_5_data/report-count")

In [3]:
result_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- count: long (nullable = true)



In [3]:
result_df.orderBy(F.desc("count")).show()

+----------+-----------+-----+
|product_id|review_date|count|
+----------+-----------+-----+
|B000NZW3IY| 2009-05-19|   44|
|B000NZW3IY| 2009-05-21|   44|
|B000NZW3IY| 2009-05-20|   32|
|B000NZW3IY| 2009-05-22|   30|
|B002HJ377A| 2013-04-24|   28|
|B000NZW3IY| 2009-05-06|   20|
|B0053O1KHA| 2014-11-03|   17|
|B000NZW3IY| 2009-05-23|   17|
|B000NZW3IY| 2009-05-26|   12|
|B0053O1KHA| 2014-11-04|   12|
|B0053O1KHA| 2014-10-31|   11|
|B002LC72P6| 2009-10-18|    9|
|B00GXYE9AE| 2013-12-09|    8|
|B002LC72P6| 2009-10-17|    8|
|B000NZW3IY| 2009-05-24|    7|
|B000NZW3IY| 2009-05-07|    7|
|B0053O1KHA| 2014-11-01|    7|
|B002HJ377A| 2009-07-17|    7|
|B00W0R8FYE| 2015-04-13|    7|
|B00GWNHZ2K| 2013-12-01|    7|
+----------+-----------+-----+
only showing top 20 rows



# RDD

In [4]:
df.select( 
    F.col('review_headline'),
    F.col('review_body')
).limit(10).show()

+--------------------+--------------------+
|     review_headline|         review_body|
+--------------------+--------------------+
|          Four Stars|Good quality. Rea...|
|          Five Stars|It's very cute an...|
|Cute and nicely p...|I wasn't sure wha...|
|        Good quality|Very good quality...|
|Better to buy tha...|Very well made an...|
|          Five Stars|Love it!!! Fits a...|
|I ordered the 14 ...|Very cute but was...|
|Good quality for ...|My boy looks grea...|
|           Two Stars|The sleeves are w...|
|No dissappointmen...|Great buy! Got th...|
+--------------------+--------------------+



In [5]:
rdd_review_body = df.select("product_id","review_body").rdd

In [17]:
rdd_review_body.take(2)

[Row(product_id='B00JA7GSVO', review_body='Good quality. Reasonable price.<br />Thank you for quick delivery.'),
 Row(product_id='B00JA7GSO6', review_body="It's very cute and just like I expected!")]

In [162]:
all_data = rdd_review_body.collect()

In [10]:
rdd_review_body.getNumPartitions()

9

In [11]:
rdd_review_body.repartition(4).getNumPartitions()

4

In [12]:
rdd_review_body.getNumPartitions()

9

### Map

In [23]:
def count_len(row):
    review = row.review_body
    product_id = row.product_id
    return (product_id,len(review))

def count_len_partition(idx, partitionRows):
    for row in partitionRows:
        review = row['review_body']
        product_id = row['product_id']
        yield (idx, product_id,len(review))

In [24]:
rdd_review_body.map(count_len).take(10)

[('B00JA7GSVO', 66),
 ('B00JA7GSO6', 40),
 ('B00JA7GSO6', 223),
 ('B00JA7GS3W', 61),
 ('B00JA7GS3W', 199),
 ('B00JA7GS3W', 62),
 ('B00JA7GS3W', 131),
 ('B00JA7GS3W', 50),
 ('B00JA7GS3W', 30),
 ('B00JA7GS3W', 117)]

In [25]:
rdd_review_body.mapPartitionsWithIndex(count_len_partition).take(5)

[(0, 'B00JA7GSVO', 66),
 (0, 'B00JA7GSO6', 40),
 (0, 'B00JA7GSO6', 223),
 (0, 'B00JA7GS3W', 61),
 (0, 'B00JA7GS3W', 199)]

In [31]:
def count_word(row):
    review = row.review_body
    for word in review.split(" "):
        yield (word,1)

In [33]:
rdd_review_body.flatMap(count_word).take(10)

[('Good', 1),
 ('quality.', 1),
 ('Reasonable', 1),
 ('price.<br', 1),
 ('/>Thank', 1),
 ('you', 1),
 ('for', 1),
 ('quick', 1),
 ('delivery.', 1),
 ("It's", 1)]

In [34]:
def filter_product(row):
    product_id = row.product_id
    return product_id != "B01KL6O72Y"

In [35]:
( 
    rdd_review_body
        .filter(filter_product)
        .map(count_len)
        .take(10)
)

[('B00JA7GSVO', 66),
 ('B00JA7GSO6', 40),
 ('B00JA7GSO6', 223),
 ('B00JA7GS3W', 61),
 ('B00JA7GS3W', 199),
 ('B00JA7GS3W', 62),
 ('B00JA7GS3W', 131),
 ('B00JA7GS3W', 50),
 ('B00JA7GS3W', 30),
 ('B00JA7GS3W', 117)]

In [36]:
def sort_by_value(record):
    return record[1]

In [37]:
(
    rdd_review_body
        .filter(filter_product)
        .map(count_len)
        .sortBy(sort_by_value,ascending = False)
        .take(10)
)

[('B00ACT5AK4', 30540),
 ('B000NZW3IY', 25911),
 ('B00GMSWYVC', 23369),
 ('B005304YG8', 18598),
 ('B006N1KN9I', 16336),
 ('B00WDZ81JC', 16332),
 ('B00O7AS0MY', 14273),
 ('B00188OAZO', 14262),
 ('B000PGVZFI', 14262),
 ('B003MQN5P6', 14262)]

### Reduce

In [39]:
def my_sum(x,y):
    return x + y

In [41]:
(
    rdd_review_body
        .map(count_len)
        .reduceByKey(my_sum)
        .sortBy(sort_by_value,ascending = False)
        .take(10)
)

[('B002HJ377A', 1239509),
 ('B000NZW3IY', 1014301),
 ('B004M6XUI2', 363297),
 ('B006PGGJOE', 352309),
 ('B004M6UDF0', 346183),
 ('B0045H0L1W', 314670),
 ('B004M6UD46', 252959),
 ('B001PU9A9Q', 250071),
 ('B00FJ5LI3O', 224750),
 ('B007NOEWV4', 221767)]

In [181]:
from collections import namedtuple
ProductWordCountRow = namedtuple("ProductWordCountRow",["product_id","len_count"])

def return_to_row(record):
    return ProductWordCountRow(record[0],record[1])

df = (
    rdd_review_body
        .map(count_len)
        .reduceByKey(my_sum)
        .map(return_to_row)
        .toDF()
)

In [182]:
df.show()

+----------+---------+
|product_id|len_count|
+----------+---------+
|B01I497BGY|       87|
|B01G6MBEBY|       51|
|B01FWRXN0Y|       48|
|B01DXHX81O|       10|
|B01DDULIJK|      197|
|B01B3Q4Q0O|       93|
|B01ADDSL9U|       10|
|B019MDXIXG|       10|
|B019438FEG|       59|
|B0178HGNIA|       62|
|B016VIU0QI|      366|
|B016PUU3VO|      231|
|B016AQNDM4|       19|
|B01694YS8K|      135|
|B015YCHLHS|       83|
|B014WCV7JY|       21|
|B014PKNCGE|       38|
|B014MSSP66|       40|
|B014L79H8I|      461|
|B014K3PHXW|       33|
+----------+---------+
only showing top 20 rows



In [49]:
def gen_word(row):
    '''Returns an iterator of words for a row'''
    for word in row['review_body'].split(" "):
        yield word, 1

# groupByKey and map can be combined using reduceByKey
# like below!
rdd_review_body \
    .flatMap(gen_word) \
    .groupByKey() \
    .map(lambda x: (x[0], sum(x[1]))) \
    .sortBy(lambda x: x[1], ascending=False) \
    .take(10)

# Word Count  bằng rdd

In [1]:
rdd_review_body \
    .flatMap(count_word) \
    .reduceByKey(lambda x,y: x + y) \
    .sortBy(lambda x: x[1], ascending=False) \
    .take(10)

NameError: name 'rdd_review_body' is not defined

In [184]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

def count_word(row):
    review = row.review_body
    for word in review.split(" "):
        yield (word,1)

def my_sum(x,y):
    return x + y

spark = ( 
    SparkSession.builder
            .master("yarn") 
            .appName('Aggreate Good Comment')
            .getOrCreate()
)

schema = types.StructType([
    types.StructField('marketplace', types.StringType(), True),
    types.StructField('customer_id', types.StringType(), True),
    types.StructField('review_id', types.StringType(), True),
    types.StructField('product_id', types.StringType(), True),
    types.StructField('product_parent', types.StringType(), True),
    types.StructField('product_category', types.StringType(), True),
    types.StructField('star_rating', types.IntegerType(), True),
    types.StructField('helpful_votes', types.IntegerType(), True),
    types.StructField('total_votes', types.IntegerType(), True),
    types.StructField('vine', types.LongType(), True),
    types.StructField('verified_purchase', types.LongType(), True),
    types.StructField('review_headline', types.StringType(), True),
    types.StructField('review_body', types.StringType(), True),
    types.StructField('review_date', types.StringType(), True)
])

data_storage = "gs://aws-review-data/amazon_us_reviews-train-00000-of-00005.parquet"
data_write = "gs://aws-review-data/write/world-count"

df = spark.read.schema(schema).parquet(data_storage)
rdd_review_body = df.select("review_body").rdd
(
    rdd_review_body
        .flatMap(count_word)
        .reduceByKey(my_sum)
        .saveAsTextFile(data_write)
)
